In [ ]:
import os
print(os.path.abspath(os.curdir))
os.chdir("..")
print(os.path.abspath(os.curdir))
import json
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from request import Request, run_request
from langchain_ollama import ChatOllama
from sklearn.utils import shuffle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils import keep_numbers

In [2]:
df = pd.read_csv("data/organisciak_2023/all_data.csv")
df

,n_id,type,src,question,prompt,response,id,target,participant,response_num
0,453,uses,hass17,What is a surprising use for a BOTTLE?,bottle,to catch an insect in,hass17_bottle-8f9afe,3.7,hass1737,7.0
1,180,instances,setal08,What is a surprising thing that makes a NOISE?,noise,stomach (hungry),setal08_5.0-7905ba,1.3,setal085,1.0
2,1966,uses,hmsl,What is a surprising use for a PAPERCLIP?,paperclip,etch on a surface,hmsl_paperclip-f58dc8,3.0,hmslLr6WcnQu,4.0
3,11185,instances,setal08,What is a surprising thing that is ROUND?,round,toy,setal08_2.0-641dd1,1.0,setal08234,1.0
4,2862,uses,betal18,What is a surprising use for a ROPE?,rope,bracelet,betal18_rope-580881,1.0,betal182183,NaN
...,...,...,...,...,...,...,...,...,...,...
33107,3966,uses,snbmo09,What is a surprising use for a BOX?,box,make a design and put it on wall,snbmo09_3_box-3ff700,1.7,snbmo09194,5.0
33108,1094,uses,bs12,What is a surprising use for a BRICK?,brick,Throw in a fit of rage,bs12_brick-b0cd04,1.8,bs1280,NaN
33109,6385,uses,setal08,What is a surprising use for a KNIFE?,knife,cut grass,setal08_4.0-14e90b,1.3,setal08136,7.0
33110,1761,uses,hmsl,What is a surprising use for a PAPERCLIP?,paperclip,antina,hmsl_paperclip-a96241,2.5,hmsl344dbgyl,2.0


In [8]:
uses_dataset = df[df["type"] == "uses"]
uses_dataset = uses_dataset.dropna(subset=["target"])
uses_dataset = uses_dataset[uses_dataset["target"] != 0]
uses_dataset

,n_id,type,src,question,prompt,response,id,target,participant,response_num
0,453,uses,hass17,What is a surprising use for a BOTTLE?,bottle,to catch an insect in,hass17_bottle-8f9afe,3.7,hass1737,7.0
2,1966,uses,hmsl,What is a surprising use for a PAPERCLIP?,paperclip,etch on a surface,hmsl_paperclip-f58dc8,3.0,hmslLr6WcnQu,4.0
4,2862,uses,betal18,What is a surprising use for a ROPE?,rope,bracelet,betal18_rope-580881,1.0,betal182183,NaN
5,2098,uses,betal18,What is a surprising use for a ROPE?,rope,jump,betal18_rope-3bfdf9,1.0,betal182065,NaN
6,1379,uses,dod20,What is a surprising use for a SHOE?,shoe,make design in the sand,dod20_shoe-52d1c5,3.0,dod2013,10.0
...,...,...,...,...,...,...,...,...,...,...
33107,3966,uses,snbmo09,What is a surprising use for a BOX?,box,make a design and put it on wall,snbmo09_3_box-3ff700,1.7,snbmo09194,5.0
33108,1094,uses,bs12,What is a surprising use for a BRICK?,brick,Throw in a fit of rage,bs12_brick-b0cd04,1.8,bs1280,NaN
33109,6385,uses,setal08,What is a surprising use for a KNIFE?,knife,cut grass,setal08_4.0-14e90b,1.3,setal08136,7.0
33110,1761,uses,hmsl,What is a surprising use for a PAPERCLIP?,paperclip,antina,hmsl_paperclip-a96241,2.5,hmsl344dbgyl,2.0


In [ ]:
# Shuffle & split: 80% train, 10% val, 10% test
train_df, temp_df = train_test_split(
    uses_dataset,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    shuffle=True,
    random_state=42
)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [19]:
export_train_df = []
export_val_df = []
export_test_df = []

for _, row in train_df.iterrows():
    export_train_df.append({
        "prompt": str(row["prompt"]),
        "response": str(row["response"]),
        "target": str(int(row["target"]*10)),
    })

for _, row in val_df.iterrows():
    export_val_df.append({
        "prompt": str(row["prompt"]),
        "response": str(row["response"]),
        "target": str(int(row["target"]*10)),
    })

for _, row in test_df.iterrows():
    export_test_df.append({
        "prompt": str(row["prompt"]),
        "response": str(row["response"]),
        "target": str(int(row["target"]*10)),
    })

In [20]:
with open("./data/gemma3n_finetune/train.json", "w", encoding="utf-8") as f:
    json.dump(export_train_df, f, ensure_ascii=False, indent=4)

with open("./data/gemma3n_finetune/val.json", "w", encoding="utf-8") as f:
    json.dump(export_val_df, f, ensure_ascii=False, indent=4)
    
with open("./data/gemma3n_finetune/test.json", "w", encoding="utf-8") as f:
    json.dump(export_test_df, f, ensure_ascii=False, indent=4)

This is the format of the prompt I will be using to finetune gemma. Therefore, it also has to be use for inference.

This is the chat template from Gemma 3n:

```
<bos><start_of_turn>user\nHello!<end_of_turn>\n<start_of_turn>model\nHey there!<end_of_turn>\n<start_of_turn>user\nWhat is 1+1?<end_of_turn>\n<start_of_turn>model\n
```

In [ ]:
def format_prompt(example):
    return f'<bos><start_of_turn>user\nYou are an expert alternative uses test (AUT) rater. The following is a response of a creative or surprising use of a {example["prompt"].upper()}. On a scale of 10-50, judge how original this use for {example["prompt"].upper()} is, where 10 is "not at all creative" and 50 is "very creative". To rate:"{example["response"]}"<end_of_turn>\n<start_of_turn>model\n{example["target"]}<end_of_turn>\n'